## Mean Absolute Scaled Error
We want to track, during Optuna Trials / mlflow experiments, also the [Mean Absolute Scaled Error](https://en.wikipedia.org/wiki/Mean_absolute_scaled_error) (MASE). Since we are renewing the requirements file including [sktime](https://www.sktime.net/en/stable/index.html), we use it to track all metrics

In [1]:
from sktime.performance_metrics.forecasting import MeanAbsoluteScaledError, MeanAbsolutePercentageError, MeanAbsoluteError, MeanSquaredError

In [2]:
import numpy as np 
import pandas as pd 

from enefit_challenge.utils.dataset import load_enefit_training_data

In [3]:
import numpy as np 
import pandas as pd 
import mlflow
import mlflow.lightgbm
from mlflow.models import infer_signature
from mlflow.tracking import MlflowClient
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from lightgbm import LGBMRegressor

import optuna
from optuna.integration.mlflow import MLflowCallback
import joblib

from typing import Optional, Dict, Tuple, Literal
from enefit_challenge.models.forecaster import Forecaster

import warnings
warnings.filterwarnings('ignore')

TRACKING_URI = "http://127.0.0.1:5000/" # local tracking URI -> launch mlflow before training 

/Users/dylantartarini/miniconda3/envs/enefit/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
class LightGBMForecaster(Forecaster):
    """
        Implementation of a Forecaster using `LGBMRegressor` as base model, 
        `optuna` for hyperparameters optimization and `mlflow` as backend to track experiments
        and register best-in-class model for time series prediction.
    """
    def __init__(self)-> None:
        self.tracking_uri = mlflow.set_tracking_uri(TRACKING_URI)
        pass

    def fit_model(
        self,  
        X:pd.DataFrame,
        y:pd.Series,
        params:Optional[Dict]=None,
    ) -> LGBMRegressor:
        """
        Trains a `LGBMRegressor`

        -------     
        params:
        -------
        `X`:`pd.DataFrame`
            Features to use for fitting
        `y`:`pd.Series`
            Target variable
        `params`: `Optional[Dict]`
            optional dictionary of parameters to use
        -------     
        returns:
        -------
        fitted `LGBMRegressor`
        """
        model = LGBMRegressor(
            n_estimators=100, 
            objective='regression',
        )
        if params:
            model.set_params(**params)

        model.fit(X, y)
    
        return model
    
    def fit_and_test_fold(
        self, 
        params:Dict,
        X: pd.DataFrame, 
        y: pd.Series, 
        year_month_train, 
        year_month_test,
        experiment_name: str="lightgbm",
        artifact_path: str="lightgbm_model",
        metrics: list=["mae"]
    ) -> float:
        """
        Used for cross validation on different time splits; 
        also in charge of logging every experiment run / study trial into the backend.
        """
        
        first_dates_month = pd.to_datetime(X[['year', 'month']].assign(day=1))
        train_index = first_dates_month.isin(year_month_train)
        test_index = first_dates_month.isin(year_month_test)

        X_train = X[train_index]
        X_test = X[test_index]
        y_train = y[train_index]
        y_test = y[test_index]

        # fit model on training data
        mlflow.lightgbm.autolog(log_models=False, log_datasets=False)
        model = self.fit_model(
            X_train, 
            y_train, 
            params
        )
        
        # generate predictions
        y_test_pred = model.predict(X_test)
        self.signature = infer_signature(X_train, y_test_pred)
        
        MAE = MeanAbsoluteError()
        mae = MAE(y_test, y_test_pred)
        MASE = MeanAbsoluteScaledError()
        mase = MASE(y_test, y_test_pred, y_train=y_train)
        MAPE = MeanAbsolutePercentageError()
        mape = MAPE(y_test, y_test_pred)
        MSE = MeanSquaredError()
        mse = MSE(y_test, y_test_pred)
        RMSE = MeanSquaredError(square_root=True)
        rmse = RMSE(y_test, y_test_pred)

        mlflow.lightgbm.log_model(
            model, 
            artifact_path=artifact_path,
            signature=self.signature
        )
        mlflow.log_params(params)

        return mae, mase, mse, rmse, mape

    def train_model(
        self, 
        train_df: pd.DataFrame, 
        target_col: str,
        model_name: str,
        exclude_cols: list=[],
        categorical_features: list=[],
        experiment_name: str="lightgbm",
        artifact_path: str="lightgbm_model",
        params: Optional[Dict]=None
    ) -> None:
        """ 
        Takes an instance of `LGBMRegressor` model and tracks the hyperparameter tuning
        experiment on training set using `mlflow` and `optuna`.  
        Registers the best version of the model according to a specified metric (to be implemented).
        
        -------     
        params:
        -------
        `experiment_name`: `str`
            the name of the experiment used to store runs in mlflow, 
            as well as the name of the optuna study
        `model_name`: `str`
            the name the final model will have in the registry
        `train_df`: `pd.DataFrame`
            the training data for the model.
        `target_col`: `str`
            the time-series target column
        `exclude_cols`: `list`  
            columns in dataset that should not be used
        `categorical_features`: `list`
            list of categorical features in the dataset
        `artifact_path`: `str`
            the path pointing to the mlflow artifact
        `params`: `Optional[Dict]`
            optional dictionary of parameters to use
        """
        self.model_name = model_name

        if len(categorical_features) > 0: 
           train_df = pd.get_dummies(train_df, columns=categorical_features)

        X = train_df.drop([target_col] + exclude_cols, axis=1)
        y = train_df[target_col]
        # unique year-month combinations -> to be used in cross-validation
        timesteps = np.sort(np.array(
            pd.to_datetime(X[['year', 'month']].assign(day=1)).unique().tolist()
        ))
        
        # define mlflow callback Handler for optuna 
        mlflc = MLflowCallback(
            metric_name=["MAE"]
        )
    
        @mlflc.track_in_mlflow() # decorator to allow mlflow logging
        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, 250, log=True),
                'boosting_type': trial.suggest_categorical("boosting_type", ["gbdt", "dart"]),
                'learning_rate': trial.suggest_float('eta', 0.01, 0.95,log=True),
                'max_depth': trial.suggest_int('max_depth', 1, 10, log=True),
                'min_child_weight': trial.suggest_float('min_child_weight', 0.001, 10, log=True),
                'colsample_bytree': trial.suggest_float("colsample_bytree", 0.1, 1, log=True),
                'subsample': trial.suggest_float("subsample", 0.5, 1, log=True),
                'reg_alpha': trial.suggest_float('lambda', 1e-3, 10.0, log=True), #L1
                'reg_lambda': trial.suggest_float('alpha', 1e-3, 10.0, log=True) #L2
            }

            cv = TimeSeriesSplit(n_splits=3) # cross validation
            cv_mae = [None]*3
            cv_mase = [None]*3
            cv_mse = [None]*3
            cv_rmse = [None]*3
            cv_mape = [None]*3
            for i, (train_index, test_index) in enumerate(cv.split(timesteps)):
                cv_mae[i], cv_mase[i], cv_mse[i], cv_rmse[i], cv_mape[i] = self.fit_and_test_fold(
                    params,
                    X, 
                    y, 
                    timesteps[train_index], 
                    timesteps[test_index]
                )
            trial.set_user_attr('split_mae', cv_mae)
            trial.set_user_attr('split_mae', cv_mase)
            trial.set_user_attr('split_mape', cv_mse)
            trial.set_user_attr('split_rmse', cv_rmse)
            trial.set_user_attr('split_mae', cv_mape)

            mlflow.log_metrics(
                {
                    "MAE":np.mean(cv_mae),
                    "MASE": np.mean(cv_mase),
                    "MSE": np.mean(cv_mse),
                    "RMSE":np.mean(cv_rmse),
                    "MAPE":np.mean(cv_mape)
                }
            )
            return np.mean(cv_mae) 

        
        sampler = optuna.samplers.TPESampler(
            n_startup_trials=10, 
            seed=0
        )

        self.study = optuna.create_study(
            directions=['minimize'],
            sampler=sampler,
            study_name=experiment_name
        )

        self.study.optimize(objective, n_trials=20, timeout= 7200, callbacks=[mlflc]) 
        
        # # search for the best run at the end of the experiment # not implemented now bc of callback bug
        # best_run = mlflow.search_runs(max_results=1,order_by=["metrics.MAE"]).run_id
        # # register new model version in mlflow
        # self.result = mlflow.register_model(
        #     model_uri=f"runs:/{best_run}/{artifact_path}",
        #     name=self.model_name
        # )

    def forecast(
        self, 
        input_data: pd.DataFrame,
        use_best_from_run: bool=True,
        use_env_model: Literal["Staging", "Production", None]=None,
        use_version: int=None
        ) -> pd.DataFrame:
        """ 
        Fetches a version of the model from the mlflow backend and uses it
        to perform prediction on new input data.  
        What version is used depends on params settings, 
        defaults to using the best version from the last experiment run (currently not implemented). 
        -------     
        params:
        -------
        `input_data`: `pd.DataFrame`
            the input data for prediction,
              must have the same schema as what's in the model's signature.
        `use_best_from_run`: `bool=True`      
            use the best model from the current series of iterations, defaults to True
        `use_env_model`: `Literal["Staging", "Production", None]=None`
            use model from a given mlflow environment, defaults to None.  
            Said model might come from past iterations, depending on what you decide in the UI
        `use_version`: `int=None`
            use a previously trained version of the model. 
            Said version must have been registered from a previous iteration,  
            either by the UI or with mlflow's API
        """
        if use_best_from_run:
            # not implemented now bc of callback bug
            use_prod_model=None
            use_version=None
        
            # model = mlflow.pyfunc.load_model(
            #     model_uri=f"models:/{self.model_name}/{self.result.version}"
            # )
            # y_pred = model.predict(input_data)
            # return y_pred
        
        if use_env_model is not None:
            use_version = None

            model = mlflow.pyfunc.load_model(
                # get registered model in given environment
                model_uri=f"models:/{self.model_name}/{use_env_model}"
            )
            y_pred = model.predict(input_data)
            return y_pred

        if use_version is not None:
            # get specific registered version of model
            model = mlflow.pyfunc.load_model(
                model_uri=f"models:/{self.model_name}/{use_version}"
            )
            y_pred = model.predict(input_data)
            return y_pred

        
        if (not use_best_from_run) & (use_env_model is None) & (use_version is None):
            return ValueError(
                    "You must specify which kind of LightGBMForecaster you intend to use for prediction"
            )

In [5]:
train_df = load_enefit_training_data()
not_feature_columns = ['datetime', 'row_id','prediction_unit_id','date','time', 'data_block_id']
cat_columns = ['county', 'product_type']

In [8]:
lgbf = LightGBMForecaster()

lgbf.train_model(
   train_df=train_df,
   target_col="target",
   model_name="lightgbm_enefit",
   exclude_cols=not_feature_columns,
   categorical_features=cat_columns
)

[I 2023-12-03 18:23:29,661] A new study created in memory with name: lightgbm


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016419 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 14082
[LightGBM] [Info] Number of data points in the train set: 551778, number of used features: 91
[LightGBM] [Info] Start training from score 242.074828
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.029923 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 14486
[LightGBM] [Info] Number of data points in the train set: 1042444, number of used features: 91
[LightGBM] [Info] Start training from score 247.723599
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.044571 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is 

[I 2023-12-03 18:23:48,756] Trial 0 finished with value: 73.71418821555197 and parameters: {'n_estimators': 121, 'boosting_type': 'gbdt', 'eta': 0.11957168053633911, 'max_depth': 2, 'min_child_weight': 0.38333321561566636, 'colsample_bytree': 0.2738969595234697, 'subsample': 0.927727492754704, 'lambda': 7.155682161754866, 'alpha': 0.03417952912061012}. Best is trial 0 with value: 73.71418821555197.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007985 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 14082
[LightGBM] [Info] Number of data points in the train set: 551778, number of used features: 91
[LightGBM] [Info] Start training from score 242.074828
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022143 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 14486
[LightGBM] [Info] Number of data points in the train set: 1042444, number of used features: 91
[LightGBM] [Info] Start training from score 247.723599
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.028689 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is 

[I 2023-12-03 18:24:10,508] Trial 1 finished with value: 98.57322394762025 and parameters: {'n_estimators': 179, 'boosting_type': 'dart', 'eta': 0.6769776337422189, 'max_depth': 1, 'min_child_weight': 0.002231090560744304, 'colsample_bytree': 0.10476552591086516, 'subsample': 0.8904582312865974, 'lambda': 1.2960656597279734, 'alpha': 3.020289640158666}. Best is trial 0 with value: 73.71418821555197.


...

[I 2023-12-03 18:30:34,659] Trial 17 finished with value: 60.544727458003685 and parameters: {'n_estimators': 106, 'boosting_type': 'gbdt', 'eta': 0.08393905717296767, 'max_depth': 5, 'min_child_weight': 1.4281764499281888, 'colsample_bytree': 0.7373959768498988, 'subsample': 0.8106765540527653, 'lambda': 0.004575862849554225, 'alpha': 0.12256203967901345}. Best is trial 16 with value: 58.224452640637615.


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.019131 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 14082
[LightGBM] [Info] Number of data points in the train set: 551778, number of used features: 91
[LightGBM] [Info] Start training from score 242.074828
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furthe

[I 2023-12-03 18:30:58,200] Trial 18 finished with value: 62.91727561699176 and parameters: {'n_estimators': 105, 'boosting_type': 'gbdt', 'eta': 0.09665959649135596, 'max_depth': 5, 'min_child_weight': 0.07062985958192339, 'colsample_bytree': 0.4964142243275759, 'subsample': 0.8147110605200386, 'lambda': 0.005802947889603856, 'alpha': 0.14347812786996073}. Best is trial 16 with value: 58.224452640637615.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.033637 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 14082
[LightGBM] [Info] Number of data points in the train set: 551778, number of used features: 91
[LightGBM] [Info] Start training from score 242.074828
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.052475 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_w

[I 2023-12-03 18:31:21,396] Trial 19 finished with value: 64.91670455839802 and parameters: {'n_estimators': 145, 'boosting_type': 'gbdt', 'eta': 0.08764215980697401, 'max_depth': 3, 'min_child_weight': 1.1809009933574914, 'colsample_bytree': 0.686038316280143, 'subsample': 0.8343042856666202, 'lambda': 0.0059918731204437825, 'alpha': 0.11973116630505895}. Best is trial 16 with value: 58.224452640637615.
